# Indicaciones

## A. Objetivos de la tarea

El objetivo de esta tarea es desarrollar y aplicar un modelo de machine learning para predecir el número de personas que no asisten a abordar los vuelos de la aerolínea AeroML, basándose en distintos parámetros como ruta, día de la semana, hora del día, entre otros.

## B. Prerrequisitos para desarrollar la tarea

Antes de trabajar en esta tarea deben haber comprendido los contenidos de la unidad de sobre algoritmos de aprendizaje supervisado, especialmente la temática sobre árboles de decisión.

## C. Instrucciones para la elaboración de la tarea

REQUISITOS GENERALES

1. **Análisis exploratorio de los datos**<p>
2. **Preparación de los datos**<p>
3. **Extracción de atributos de la fecha**<p>
4. **Selección de variables**<p>
5. **Desarrollo del modelo**<p>
6. **Reflexione sobre el modelo**<p>
7. **Evaluación del modelo**<p>
8. **Reflexión final**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Cargar los datos
data = pd.read_csv("datos_vuelos_AeroML.csv", parse_dates=["fecha", "hora_salida"])

## TIP: Puede extraer información de las fechas de la siguiente manera:

```python

data['año'] = data['fecha'].dt.year

## Descripción de Variables en el Conjunto de Datos de Vuelos

### Variables Principales

1. **`id`**:  
   - Identificador único para cada registro.
   - Útil para indexación y combinación de datos.
  
2. **`fecha`**:  
   - Fecha del vuelo.
   - Importante para el análisis temporal y la identificación de patrones estacionales o tendencias.
  
3. **`numero_vuelo`**:  
   - Identificador único del vuelo.
   - Sirve para rastrear la eficiencia y popularidad de rutas específicas.
  
4. **`origen` y `destino`**:  
   - Aeropuertos de origen y destino.
   - Pueden usarse para analizar la demanda entre diferentes ubicaciones.

5. **`distancia`**:  
   - Distancia entre el origen y el destino.
   - Puede influir en la tarifa y el consumo de combustible.

6. **`capacidad`**:  
   - Número total de asientos en el avión.
   - Útil para calcular la tasa de ocupación y eficiencia del vuelo.

7. **`venta_usd`**:  
   - Ingresos totales del vuelo en dólares estadounidenses.
   - Un KPI clave para la rentabilidad.

8. **`agendados`**:  
   - Número total de reservas para el vuelo.
   - Indica la demanda y permite calcular la tasa de ocupación.

### Variables de Pasajeros

9. **`inasistencia`**:  
   - Número de pasajeros que no se presentaron.
   - Esto puede afectar la rentabilidad y requiere estrategias de overbooking cuidadosas.

10. **`vuelo_denegado`**:  
    - Número de pasajeros que no pudieron abordar debido al exceso de reservas.
    - Importante para evaluar la eficiencia del algoritmo de overbooking.

### Variables de Tarifas

11. **`tarifa_mediabaja`, `tarifa_alta`, `tarifa_mediaalta`, `tarifa_baja`**:  
    - Distribución de los tipos de tarifas adquiridas.
    - Esto puede ayudar a segmentar a los clientes y ajustar las estrategias de precios.

12. **`pax_freqflyer`**:  
    - Número de pasajeros que redimieron millas.
    - Importante para medir el compromiso del cliente.

13. **`agendado_grupal`**:  
    - Número de reservas grupales.
    - Estos suelen tener tarifas más bajas y podrían afectar la rentabilidad.

### Variables de Conexión

14. **`conexion_nacional`, `conexion_internacional`, `sin_conexion`**:  
    - Indican si el vuelo es parte de una conexión nacional, internacional o si los pasajeros no están en una conexión.
    - Esto puede afectar la logística y la planificación.

### Variables Adicionales

15. **`sin_stock`**:  
    - Días sin capacidad para vender más boletos.
    - Un indicador de alta demanda que podría usarse para ajustar tarifas o frecuencias de vuelo.

16. **`year`, `month`, `day`, `day_of_week`, `hour`**:  
    - Variables temporales que pueden ser útiles para modelar efectos estacionales o patrones diurnos.


# 1. Análisis Exploratorio y Preparación de Datos (se evalúa acá pero no importa si se realiza más adelante)

Realice un análisis exploratorio y una preparación de datos que incluya:
- La gestión de valores faltantes o nulos (si existieran).
- La codificación de variables categóricas (si existieran).
- La normalización o estandarización de los datos.

**Comentarios:**
- Comente si existen datos faltantes, si hay valores que no tienen sentido, si hay outliers, etc.
- Describa las decisiones que tomó para la limpieza, imputación y/o transformación de los datos.

### Análisis exploratorio básico

In [2]:
data.head(3)

,id,fecha,numero_vuelo,origen,destino,distancia,inasistencia,vuelo_denegado,tarifa_mediabaja,tarifa_alta,...,pax_freqflyer,agendado_grupal,sin_stock,conexion_nacional,conexion_internacional,sin_conexion,hora_salida,capacidad,venta_usd,agendados
0,69922,2009-02-23,8942,ANF,SCL,1106,7,0,124.0,5,...,20,0,0,0,0,259.0,2024-06-19 21:15:00,168,8399.7,259.0
1,469723,2010-01-13,8941,SCL,ANF,1106,18,0,56.0,0,...,4,0,0,7,6,96.0,2024-06-19 18:15:00,174,8535.7,109.0
2,779308,2010-10-04,9128,ANF,SCL,1106,6,0,1.0,0,...,2,0,0,0,1,78.0,2024-06-19 17:26:00,218,3525.4,79.0


In [3]:
data.shape

(18322, 22)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18322 entries, 0 to 18321
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      18322 non-null  int64         
 1   fecha                   18322 non-null  datetime64[ns]
 2   numero_vuelo            18322 non-null  int64         
 3   origen                  18322 non-null  object        
 4   destino                 18322 non-null  object        
 5   distancia               18322 non-null  int64         
 6   inasistencia            18322 non-null  int64         
 7   vuelo_denegado          18322 non-null  int64         
 8   tarifa_mediabaja        17315 non-null  float64       
 9   tarifa_alta             18322 non-null  int64         
 10  tarifa_mediaalta        18322 non-null  int64         
 11  tarifa_baja             18322 non-null  int64         
 12  pax_freqflyer           18322 non-null  int64 

In [5]:
data.describe()

,id,numero_vuelo,distancia,inasistencia,vuelo_denegado,tarifa_mediabaja,tarifa_alta,tarifa_mediaalta,tarifa_baja,pax_freqflyer,agendado_grupal,sin_stock,conexion_nacional,conexion_internacional,sin_conexion,capacidad,venta_usd,agendados
count,1.832200e+04,18322.000000,18322.0,18322.000000,18322.000000,17315.000000,18322.000000,18322.000000,18322.000000,18322.000000,18322.000000,18322.000000,18322.000000,18322.000000,18048.000000,18322.000000,18322.000000,17681.000000
mean,6.129082e+05,8960.270276,1106.0,8.327148,0.055180,50.279873,2.717607,4.352254,83.468781,11.110523,3.858804,0.072863,10.299858,4.090929,137.524601,185.937780,10767.573158,152.004864
std,3.838625e+05,325.001168,0.0,5.443675,0.578305,49.124795,5.704688,9.669340,40.563930,7.556938,12.973262,0.259919,18.130702,6.044964,64.266551,21.487634,5968.978632,64.448983
min,1.580000e+02,7926.000000,1106.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,126.000000,193.700000,3.000000
25%,2.771338e+05,8933.000000,1106.0,5.000000,0.000000,2.000000,0.000000,0.000000,57.000000,6.000000,0.000000,0.000000,1.000000,1.000000,89.000000,174.000000,6517.850000,102.000000
50%,5.752390e+05,9126.000000,1106.0,8.000000,0.000000,36.000000,1.000000,1.000000,87.000000,10.000000,0.000000,0.000000,5.000000,2.000000,127.000000,174.000000,9595.400000,144.000000
75%,9.329845e+05,9138.000000,1106.0,11.000000,0.000000,97.000000,3.000000,4.000000,111.000000,15.000000,0.000000,0.000000,13.000000,5.000000,184.000000,218.000000,13753.200000,201.000000
max,1.350202e+06,9251.000000,1106.0,79.000000,20.000000,185.000000,109.000000,138.000000,276.000000,84.000000,158.000000,1.000000,181.000000,89.000000,366.000000,274.000000,52995.600000,384.000000


In [9]:
data.nunique()

id                        18322
fecha                      1072
numero_vuelo                125
origen                        2
destino                       2
distancia                     1
inasistencia                 60
vuelo_denegado               16
tarifa_mediabaja            177
tarifa_alta                  68
tarifa_mediaalta             90
tarifa_baja                 230
pax_freqflyer                60
agendado_grupal             125
sin_stock                     2
conexion_nacional           151
conexion_internacional       74
sin_conexion                334
hora_salida                1164
capacidad                    17
venta_usd                 17450
agendados                   329
dtype: int64

In [8]:
data.isnull().sum()

id                           0
fecha                        0
numero_vuelo                 0
origen                       0
destino                      0
distancia                    0
inasistencia                 0
vuelo_denegado               0
tarifa_mediabaja          1007
tarifa_alta                  0
tarifa_mediaalta             0
tarifa_baja                  0
pax_freqflyer                0
agendado_grupal              0
sin_stock                    0
conexion_nacional            0
conexion_internacional       0
sin_conexion               274
hora_salida                  5
capacidad                    0
venta_usd                    0
agendados                  641
dtype: int64

# 2. Selección de Variables y Extracción de Atributos

Elija las variables más relevantes para predecir la inasistencia y realice una extracción de atributos de la fecha. Explique las razones para dejar algunas características fuera.

**Comentarios:**
- Describa las variables que eligió y por qué.
- Explique el proceso de extracción de atributos de la fecha y su relevancia para el modelo.

In [ ]:
# SU CODIGO AQUI

# 3. Desarrollo y Evaluación del Modelo

Desarrolle uno o más modelos de regresión y evalúe el desempeño de los modelos mediante el error absoluto medio (MAE). Interprete correctamente los resultados y su impacto en la precisión del modelo.

**Comentarios:**
- Describa el modelo que eligió y por qué.
- Comente sobre los resultados obtenidos del MAE y su interpretación.
- Elija otra métrica de evaluación, interprete los resultados y compare con el MAE.

In [ ]:
# SU CODIGO AQUI

# 4. Reflexión Final

Reflexione sobre el proceso de desarrollo del modelo, las dificultades encontradas, las decisiones tomadas y la importancia de la evaluación y selección de modelos en la creación de modelos de aprendizaje automático precisos y confiables.

**Comentarios:**
- Reflexione sobre las decisiones tomadas durante el desarrollo del modelo.
- Comente las dificultades encontradas y cómo las abordó.
- Discuta la importancia de la evaluación y selección de modelos.

In [ ]:
# SU CODIGO AQUI